In [1]:
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertConfig, BertTokenizer, AutoTokenizer
import datasets
from datasets import Dataset
import pandas as pd
import tensorflow as tf
import numpy as np
import nltk
import nltk.translate.meteor_score as meteor
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import keras
#import wandb
import torch
from torch.utils.data import TensorDataset, DataLoader
import pickle
import pdfplumber
from transformers import BigBirdPegasusForConditionalGeneration, TFTrainer, TFTrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, TrainingArguments, BartForConditionalGeneration, DataCollatorForSeq2Seq

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
torch.cuda
torch.cuda.is_available()

True

In [4]:
# cuda issues with this model
#path = "/home/ubuntu/Documents/OliverVinzelberg/led_large_16384_pubmed_CTS_untouched_blind_bestsofar_3072_1024_2bs_20e"

# this model works
path = "/home/ubuntu/Documents/OliverVinzelberg/led_large_16384_pubmed_CTS_untouched_3072_1024_2bs_20e"
model = AutoModelForSeq2SeqLM.from_pretrained(path, local_files_only=True)
model.cuda(1)

tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)

In [5]:
def generate_sum(data):
    to_pred = tokenizer(data, padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    #to_pred = tokenizer(data, padding="max_length", max_length=3072, return_tensors="pt", truncation=True)
    input_ids=to_pred["input_ids"].cuda(1)
    attention_mask=to_pred["attention_mask"].cuda(1)
    #global attention on special tokens
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1
    
    predicted_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
    
    return tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

In [6]:
inds = ['004', '075', '096', '124', '129', '281', '285', '337', '350', '392', '404', '475', '477']
max_input = 4096

for i in inds: 
    file = '/home/ubuntu/streamlit-example/ClinicalTrialSum/TestFiles/'+i+'_in.pdf'
    file_in = '/home/ubuntu/streamlit-example/ClinicalTrialSum/TestFiles/'+i+'_in.txt'
    file_out = '/home/ubuntu/streamlit-example/ClinicalTrialSum/TestFiles/'+i+'_out.txt'
    
    text = ''
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
            text += "\n"
    
    with open(file_in, 'w') as f:
        f.write(text)
        
    raw_text = text
    chunked_input = [raw_text[i:i+max_input] for i in range(0, len(raw_text), max_input)]
    
    output = []
    for chunk in range(0, len(chunked_input)):
        output.append(generate_sum(chunked_input[chunk]))
        
    out_str = ''
    for block in output:
        out_str += ' '+block[0]
    
    with open(file_out, 'w') as f:
        f.write(out_str)